In [41]:
print("Loading text data...")
text = open('shakespeare.txt', encoding='utf-8').read().lower()
#print('corpus length:', len(text))

Tx = 40
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Loading text data...


### Unigram

In [60]:
import numpy as np
import collections
import sys

class LaplaceBigramLanguageModel:

    def __init__(self, corpus):
        """Initialize your data structures in the constructor."""

        self.model_unigram = collections.defaultdict(int)
        self.model_bigram = collections.defaultdict(lambda : \
                collections.defaultdict(int))
        self.train(corpus)

    def train(self, corpus):
        """ Takes a corpus and trains your language model. 
        Compute any counts or other corpus statistics in this function.
    """

        last_token = '#'  # init token of sentence
        for token in corpus:
            self.model_unigram[token] += 1
            self.model_bigram[last_token][token] += 1
            last_token = token

    def score(self, sentence):
        """ Takes a list of strings as argument and returns the log-probability of the 
        sentence using your language model. Use whatever data you computed in train() here.
    """

        last_token = '#'  # init token of sentence
        p = 0
        total_vocab = len(list(self.model_unigram.keys()))
        for token in sentence:
            p += np.log2(self.model_bigram[last_token][token] + 1)
            p -= np.log2(self.model_unigram[last_token] + total_vocab)
            last_token = token
        return p
    
    def generate(self):
        usr_input = input("Nhap vao cau tho dau tien cua bai tho, Chung toi se giup ban hoan thanh bai tho:")
        begin_text = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
        
        total_vocab = list(self.model_unigram.keys())
        generated = usr_input 

        sys.stdout.write("\n\nDay la bai tho cua ban: \n\n") 
        sys.stdout.write(usr_input)
        if len(begin_text) <= 1:
            last_token = '#'
        else:
            last_token = begin_text[-1]
        for i in range(400):
            pred = np.zeros(len(total_vocab))
            for i, token in enumerate(total_vocab):
                pred[i] += np.log2(self.model_bigram[last_token][token] + 1)
                pred[i] -= np.log2(self.model_unigram[last_token] + len(total_vocab))
            index_predict = np.argmax(pred)
            token_predict = total_vocab[index_predict]
            last_token = token_predict
            generated += token_predict
            
            sys.stdout.write(token_predict)
            sys.stdout.flush()

In [61]:
model = LaplaceBigramLanguageModel(text)

In [62]:
model.generate()

Nhap vao cau tho dau tien cua bai tho, Chung toi se giup ban hoan thanh bai tho:here we are,


Day la bai tho cua ban: 

here we are,
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the

### Bigram by nltk

In [63]:
from nltk import trigrams, bigrams
from collections import defaultdict

model = defaultdict(lambda: defaultdict(lambda: 0))
print("Number of sentences:", len(text.split("\n\n\n")))
for sentence in text.split("\n\n\n"):
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
        

Number of sentences: 79


In [64]:
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [80]:
def sample(preds, temperature=1.0):
    # dua ra 1 index tu vector xac suat dau vao
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p = probas.ravel())
    return out

In [83]:
import random

def generate_trigram():
    usr_input = input("Nhap vao cau tho dau tien cua bai tho, Chung toi se giup ban hoan thanh bai tho:")
    begin_text = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
        
    total_vocab = chars
    generated = usr_input 

    sys.stdout.write("\n\nDay la bai tho cua ban: \n\n") 
    sys.stdout.write(usr_input)
    if len(begin_text) <= 1:
        last_token = None
        last_last_token = None
    elif len(begin_text) <= 2:
        last_token = begin_text[-1]
        last_last_token = None
    else:
        last_token = begin_text[-2]
        last_last_token = begin_text[-1]
    for i in range(400):
        pred = np.zeros(len(total_vocab))
        for i, token in enumerate(total_vocab):
            pred[i] = (model[(last_last_token, last_token)][token] + 1) / (np.sum(list(model[(last_last_token, last_token)].values())) + len(total_vocab))
        index_predict = np.argmax(pred)
        token_predict = total_vocab[index_predict]
        last_last_token = last_token
        last_token = token_predict
            
        generated += token_predict
            
        sys.stdout.write(token_predict)
        sys.stdout.flush()
generate_trigram()

Nhap vao cau tho dau tien cua bai tho, Chung toi se giup ban hoan thanh bai tho:here we are,


Day la bai tho cua ban: 

here we are,
the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the